In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
import sys
sys.path.insert(0,'..')
from utils import lossprettifier
from Classifier.ResNet import *

Using TensorFlow backend.
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Aleks

In [3]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [4]:
x_test.shape

(30, 224, 224, 3)

In [5]:
#x_train /= 255????

In [6]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
#import lossprettifier
#from ResNet import *

In [7]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [8]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [9]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [10]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

C:\Users\Aleksander Podsiad\.conda\envs\WB_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [11]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating ResNet18 model
model = ResNet18((224, 224, 3), 4) 

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)


reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...

Epoch 1/50
32/32 [==============================] - 9s 283ms/step - loss: 3.8369 - accuracy: 0.6031 - val_loss: 166.7694 - val_accuracy: 0.4000
Epoch 2/50
32/32 [==============================] - 5s 150ms/step - loss: 3.2129 - accuracy: 0.9286 - val_loss: 91.9243 - val_accuracy: 0.3000
Epoch 3/50
32/32 [==============================] - 5s 148ms/step - loss: 2.9655 - accuracy: 0.9979 - val_loss: 44.9503 - val_accuracy: 0.3000
Epoch 4/50
32/32 [==============================] - 5s 149ms/step - loss: 2.9375 - accuracy: 0.9989 - val_loss: 24.7767 - val_accuracy: 0.3000
Epoch 5/50
32/32 [==============================] - 5s 150ms/step - loss: 2.9219 - accuracy: 1.0000 - val_loss: 10.7516 - val_accuracy: 0.4333
Epoch 6/50
32/32 [==============================] - 5s 151ms/step - loss: 2.9156 - accuracy: 1.0000 - val_loss: 5.6936 - val_accuracy: 0.6667
Epoch 7/50
32/32

In [12]:
modelPath = "ResNet18_COVID19.h5"
resultPath = 'ResNet18_COVID19.txt'

In [13]:
# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

y_test_oh = dense_to_one_hot(y_test,num_clases=4)

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 3.84(+0.00%) 	| LossAB: 166.77(+0.00%) 	
Epoch     1 | LossA: 3.20(-16.60%) ▼	| LossAB: 91.92(-44.88%) ▼	
Epoch     2 | LossA: 2.96(-7.42%) ▼	| LossAB: 44.95(-51.10%) ▼	
Epoch     3 | LossA: 2.94(-0.93%) ▼	| LossAB: 24.78(-44.88%) ▼	
Epoch     4 | LossA: 2.92(-0.50%) ▼	| LossAB: 10.75(-56.61%) ▼	
Epoch     5 | LossA: 2.91(-0.25%) ▼	| LossAB: 5.69(-47.04%) ▼	
Epoch     6 | LossA: 2.91(-0.30%) ▼	| LossAB: 5.46(-4.16%) ▼	
Epoch     7 | LossA: 2.91(-0.00%) ▼	| LossAB: 4.86(-10.90%) ▼	
Epoch     8 | LossA: 2.94(+1.22%) ▲	| LossAB: 4.71(-3.10%) ▼	
Epoch     9 | LossA: 2.95(+0.27%) ▲	| LossAB: 5.05(+7.14%) ▲	
Epoch    10 | LossA: 2.91(-1.44%) ▼	| LossAB: 4.41(-12.61%) ▼	
Epoch    11 | LossA: 2.89(-0.73%) ▼	| LossAB: 4.43(+0.39%) ▲	
Epoch    12 | LossA: 2.87(-0.40%) ▼	| LossAB: 4.50(+1.59%) ▲	
Epoch    13 | LossA: 2.86(-0.49%) ▼	| LossAB: 4.10(-8.85%) ▼	
Epoch    14 | LossA: 2.85(-0.35%) ▼	| LossAB: 4.21(+2.69%) ▲	
Epoch    15 | LossA: 2.84(-0.25%) ▼	| LossAB: 3.93(-6.75%)

In [14]:
y_pred = model.predict(x_test)
#y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.89      0.80      0.84        10
           2       0.78      0.70      0.74        10

    accuracy                           0.80        30
   macro avg       0.81      0.80      0.80        30
weighted avg       0.81      0.80      0.80        30



In [16]:
f.close()